In [1]:
!pip install tensorflow xgboost numpy matplotlib scikit-learn opencv-python pillow seaborn pandas setuptools


In [4]:
# Image Training only.ipynb

#  This code is for training the model



# some basic libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

# importing for basic image operations
import cv2 as cv
from PIL import Image
from PIL import ImageOps
import seaborn as sns
from datetime import datetime

# importing pretrained models
from tensorflow.keras.applications import efficientnet

# for splitting data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# prerequisites for training models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# for selecting better metrics
from sklearn.metrics import roc_auc_score



In [5]:
import tensorflow as tf

# Inicializar estratégia para treinamento
try:
    # Verifica se há GPUs disponíveis
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        # Configura o TensorFlow para usar a GPU sem alocar toda a memória (opcional)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Dispositivo detectado: {len(gpus)} GPU(s).")
        strategy = tf.distribute.MirroredStrategy()  # Estratégia para GPUs
    else:
        print("Nenhuma GPU detectada. Usando CPU.")
        strategy = tf.distribute.get_strategy()  # CPU como fallback
except RuntimeError as e:
    print(f"Erro ao configurar GPU: {e}")
    strategy = tf.distribute.get_strategy()  # Fallback para CPU

# Testando a estratégia
print("Estratégia em uso:", strategy)


Nenhuma GPU detectada. Usando CPU.
Estratégia em uso: <tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x0000020DF94A2330>


In [6]:
## Image Training

TRAIN_IMG_DIR_PATH = "../imagens_bd/results_train/"

# converts all images for transformations
X = []
y = []

#classes = ["residential", "mixed", "commercial", "others", "industrial"]
classes = ["residential", "commercial", "others", "industrial"]
INPUT_IMG_WIDTH = 250
INPUT_IMG_HEIGHT = 350
VERTICAL = True

In [7]:

print("Diretório de trabalho atual:", os.getcwd())


Diretório de trabalho atual: c:\tcc_ide\indentificacao_fachada\Scripts\notebooks


In [8]:
# Para Pillow 9.1.0 ou superior
try:
    from PIL import Resampling
    RESAMPLING_METHOD = Resampling.LANCZOS
except ImportError:
    RESAMPLING_METHOD = Image.LANCZOS

# memory management in python by some medium article asian woman

for class_no, img_class in enumerate(classes):

    for aug_index, img_file in enumerate(os.listdir(TRAIN_IMG_DIR_PATH + img_class)):

        img = Image.open(TRAIN_IMG_DIR_PATH + img_class + '/' + img_file)
        img = ImageOps.fit(img, (INPUT_IMG_WIDTH, INPUT_IMG_HEIGHT), RESAMPLING_METHOD)

        # if you want to run on no augmentation
        np_img1 = np.array(img).astype(np.float16)
        X.append(np_img1)
        y.append(class_no)

        # if you want to run on augmentation (for every 1 image, 2 additional images)
        if aug_index % 6 == 0:
            img2 = ImageOps.mirror(img)
            np_img2 = np.array(img2).astype(np.float16)
            X.append(np_img2)
            y.append(class_no)

    print(f"{img_class} pre-processing completed!")

residential pre-processing completed!
commercial pre-processing completed!
others pre-processing completed!
industrial pre-processing completed!


In [9]:
# dataset here
X = np.asarray(X)
y = np.asarray(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
del X, y

# convert to one-hot encoded vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# problem specific stuff
if VERTICAL:
    _temp_measure = INPUT_IMG_WIDTH
    INPUT_IMG_WIDTH = INPUT_IMG_HEIGHT
    INPUT_IMG_HEIGHT = _temp_measure

history_dict = []


In [10]:
# Configuração do TensorBoard
from datetime import datetime

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,  # Gera histograms
    write_graph=True,   # Grava o grafo do modelo
    write_images=True,  # Salva imagens (como visualizações de filtros)
    update_freq='epoch', # Frequência de atualização
    profile_batch=0,    # Não gera perfil de desempenho
)

In [11]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB7

# Definindo a entrada do modelo
inputs = Input(shape=(INPUT_IMG_WIDTH, INPUT_IMG_HEIGHT, 3))

# Carregando a EfficientNetB7
resnet_model = EfficientNetB7(include_top=False, input_tensor=inputs)
resnet_model.trainable = False  # Congela as camadas da EfficientNet

# Adicionando camadas superiores
x = GlobalAveragePooling2D()(resnet_model.output)
x = Dropout(0.175)(x)
x = Dense(16, activation='relu')(x)  # Camada intermediária
x = Dropout(0.125)(x)
outputs = Dense(4, activation='softmax')(x)  # Camada de saída

# Definindo o modelo
model_resnet = Model(inputs=inputs, outputs=outputs)

# Compilando o modelo
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']
)

# Treinando o modelo
history_resnet = model_resnet.fit(
    np.array(X_train), np.array(y_train),
    epochs=25,
    validation_data=(np.array(X_test), np.array(y_test)),
    batch_size=64,
    callbacks=[tensorboard_callback]
)

# Salvando o modelo
model_resnet.save("../modelo/modelo_att_2(25e).keras")


Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 42s 20s/step - accuracy: 0.3059 - auc: 0.5874 - loss: 1.3401 - val_accuracy: 0.4667 - val_auc: 0.7717 - val_loss: 1.1626
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 16s/step - accuracy: 0.5152 - auc: 0.7532 - loss: 1.1647 - val_accuracy: 0.4667 - val_auc: 0.7841 - val_loss: 1.1144
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 17s/step - accuracy: 0.5559 - auc: 0.8316 - loss: 1.0113 - val_accuracy: 0.4667 - val_auc: 0.7970 - val_loss: 1.0970
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 22s 17s/step - accuracy: 0.5331 - auc: 0.8421 - loss: 0.9619 - val_accuracy: 0.4667 - val_auc: 0.8139 - val_loss: 1.0378
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 23s 17s/step - accuracy: 0.5582 - auc: 0.8597 - loss: 0.9005 - val_accuracy: 0.5667 - val_auc: 0.8319 - val_loss: 0.9698
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 23s 17s/step - accuracy: 0.6402 - auc: 0.8936 - loss: 0.8019 - val_accuracy: 0.5667 - val_auc: 0.8474 - val_loss: 0.9149
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 23s 17s/step - accurac

In [12]:
# Salvar apenas os pesos
model_resnet.save_weights("../modelo/modelo_att.weights.h5")

# Para carregar:
# Recrie o modelo exatamente como definido acima e use `load_weights`
new_model = Model(inputs=inputs, outputs=outputs)
new_model.load_weights("../modelo/modelo_att.weights.h5")


In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Análise dos resultados a partir do `history_resnet`
val_acc = history_resnet.history['val_accuracy']
train_acc = history_resnet.history['accuracy']
val_loss = history_resnet.history['val_loss']
train_loss = history_resnet.history['loss']

# Adicionando uma linha base de acurácia inicial para comparação
train_acc = [0.25] + train_acc
val_acc = [0.25] + val_acc

# Calculando a média da perda inicial e adicionando ao início para comparação
mean_loss = (val_loss[0] + train_loss[0]) / 2
train_loss = [mean_loss] + train_loss
val_loss = [mean_loss] + val_loss

# Criando DataFrame com os dados de acurácia e perda
results_df = pd.DataFrame({
    'num': range(len(train_acc)),
    'ta': train_acc,
    'va': val_acc,
    'tl': train_loss,
    'vl': val_loss
})

# Visualizando as curvas de acurácia
plt.figure(figsize=(15, 12))
sns.lineplot(data=results_df, x='num', y='ta', label='Train Accuracy')
sns.lineplot(data=results_df, x='num', y='va', label='Validation Accuracy')
plt.title("Accuracy Curves for Building Instance Classifier")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig('../modelo/accuracy_plot_2.png')
plt.close()

# Visualizando as curvas de perda
plt.figure(figsize=(15, 12))
sns.lineplot(data=results_df, x='num', y='tl', label='Train Loss')
sns.lineplot(data=results_df, x='num', y='vl', label='Validation Loss')
plt.title("Loss Curves for Building Instance Classifier")
plt.xlabel("Epochs")
plt.ylabel("Categorical Crossentropy Loss")
plt.legend()
plt.savefig('../modelo/loss_plot_2.png')
plt.close()
